In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Step 1: Define the distance function
def calculate_distance(city1, city2):
    return np.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

# Step 2: Generate random cities
def generate_cities(num_cities):
    return np.random.rand(num_cities, 2) * 100

# Step 3: Create a class for the Genetic Algorithm
class GeneticAlgorithm:
    def __init__(self, cities, population_size=100, mutation_rate=0.01, generations=1000):
        self.cities = cities
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.population = self.initialize_population()

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = list(range(len(self.cities)))
            random.shuffle(individual)
            population.append(individual)
        return population

    def fitness(self, individual):
        total_distance = 0
        for i in range(len(individual)):
            total_distance += calculate_distance(self.cities[individual[i]], self.cities[individual[(i + 1) % len(individual)]])
        return 1 / total_distance  # Minimize distance, maximize fitness

    def select_parents(self):
        weights = [self.fitness(ind) for ind in self.population]
        return random.choices(self.population, weights=weights, k=2)

    def crossover(self, parent1, parent2):
        child = [None] * len(parent1)
        start, end = sorted(random.sample(range(len(parent1)), 2))

        # Copy the segment from parent1
        child[start:end] = parent1[start:end]

        # Fill the remaining positions from parent2
        pointer = 0
        for gene in parent2:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = gene
        return child

    def mutate(self, individual):
        for i in range(len(individual)):
            if random.random() < self.mutation_rate:
                j = random.randint(0, len(individual) - 1)
                individual[i], individual[j] = individual[j], individual[i]

    def run(self):
        best_route = None
        best_distance = float('inf')

        for generation in range(self.generations):
            new_population = []

            for _ in range(self.population_size):
                parent1, parent2 = self.select_parents()
                child = self.crossover(parent1, parent2)
                self.mutate(child)
                new_population.append(child)

            self.population = new_population
            
            # Find the best individual in the current population
            for individual in self.population:
                current_distance = 1 / self.fitness(individual)  # Get distance from fitness
                if current_distance < best_distance:
                    best_distance = current_distance
                    best_route = individual

        return best_route, best_distance

# Step 4: Plot the cities and the best route found
def plot_route(cities, route):
    plt.figure(figsize=(10, 6))
    plt.scatter(cities[:, 0], cities[:, 1], color='red')
    for i in range(len(route)):
        start_city = cities[route[i]]
        end_city = cities[route[(i + 1) % len(route)]]
        plt.plot([start_city[0], end_city[0]], [start_city[1], end_city[1]], 'b-')
    plt.title("Best Route Found")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.show()

# Step 5: Execute the algorithm
if __name__ == "__main__":
    num_cities = 10
    cities = generate_cities(num_cities)

    ga = GeneticAlgorithm(cities, population_size=200, mutation_rate=0.01, generations=1000)
    best_route, best_distance = ga.run()

    print("Best Route:", best_route)
    print("Best Distance:", best_distance)
    plot_route(cities, best_route)